# Step 1: Load Data and Set Up Libraries

In this step, we will:
1. Load the prepared JSON data for the report.
2. Import necessary libraries for generating the PDF and creating visualizations.

In [ ]:
import json
from fpdf import FPDF
import matplotlib.pyplot as plt
import os

# Load report data
input_file = "../data/processed/report_data.json"

with open(input_file, "r") as json_file:
    report_data = json.load(json_file)

# Inspect loaded data
print("Loaded report data:")
print(json.dumps(report_data, indent=4))

# Ensure output directory exists for charts
output_dir = "../outputs/charts"
os.makedirs(output_dir, exist_ok=True)

# Step 2: Chart Generation

This step generates charts and saves them as images to be used in the PDF. We will create:
1. A bar chart for sentiment distribution.
2. A pie chart for negative topic percentages.
3. Any additional charts needed for visualization.

In [ ]:
def generate_sentiment_bar_chart(sentiment_counts, output_path):
    """
    Generate a bar chart for sentiment distribution.

    Args:
        sentiment_counts (dict): Counts of positive, negative, and neutral reviews.
        output_path (str): Path to save the chart image.
    """
    sentiments = list(sentiment_counts.keys())
    counts = list(sentiment_counts.values())

    plt.figure(figsize=(8, 5))
    plt.bar(sentiments, counts, color=['green', 'red', 'gray'])
    plt.title("Sentiment Distribution")
    plt.xlabel("Sentiments")
    plt.ylabel("Number of Reviews")
    plt.savefig(output_path)
    plt.close()
    print(f"Sentiment bar chart saved to {output_path}")

def generate_negative_topic_pie_chart(negative_topic_percentages, output_path):
    """
    Generate a pie chart for negative topic percentages.

    Args:
        negative_topic_percentages (dict): Percentage distribution of negative topics.
        output_path (str): Path to save the chart image.
    """
    topics = list(negative_topic_percentages.keys())
    percentages = list(negative_topic_percentages.values())

    plt.figure(figsize=(8, 5))
    plt.pie(percentages, labels=topics, autopct='%1.1f%%', startangle=140)
    plt.title("Negative Topics Distribution")
    plt.savefig(output_path)
    plt.close()
    print(f"Negative topics pie chart saved to {output_path}")

# Generate charts
sentiment_chart_path = os.path.join(output_dir, "sentiment_distribution.png")
negative_topics_chart_path = os.path.join(output_dir, "negative_topics_distribution.png")

generate_sentiment_bar_chart(report_data["review_counts"], sentiment_chart_path)
generate_negative_topic_pie_chart(
    report_data["percentage_distribution_by_sentiment"]["negative"],
    negative_topics_chart_path
)

# Step 3: Define PDF Layout

In this step, we define a `PDFReport` class using the `fpdf` library to handle:
1. Adding a cover page with a professional design.
2. Creating headers and footers for all pages.
3. Adding sections with text and visual elements (charts and problem descriptions).

The class will be designed to embed all visualizations (charts and graphs) directly into the PDF file, ensuring a self-contained document for easy sharing and distribution.

We will generate a preview to test the layout of the cover page and a sample section.

In [ ]:
from fpdf import FPDF
import base64
from io import BytesIO
from PIL import Image
import os
from tempfile import NamedTemporaryFile


class PDFReport(FPDF):
    def header(self):
        self.set_font("Helvetica", "B", 12)
        self.cell(0, 10, "Hotel Review Analysis Report", border=False, ln=True, align="C")
        self.ln(5)

    def footer(self):
        self.set_y(-15)
        self.set_font("Helvetica", "I", 8)
        self.cell(0, 10, f"Page {self.page_no()}", align="C")

    def cover_page(self):
        self.add_page()
        self.set_font("Helvetica", "B", 20)
        self.cell(0, 80, "Hotel Review Analysis Report", ln=True, align="C")
        self.set_font("Helvetica", "", 16)
        self.cell(0, 10, "Key Insights and Recommendations for Service Excellence", ln=True, align="C")
        self.ln(20)

    def section_title(self, title):
        self.set_font("Helvetica", "B", 14)
        self.cell(0, 10, title, ln=True)
        self.ln(5)

    def section_body(self, text):
        self.set_font("Helvetica", "", 12)
        self.multi_cell(0, 10, text)
        self.ln()

    def embed_chart(self, chart_path, title=None):
        """
        Embed a chart directly into the PDF from a file path or image data.

        Args:
            chart_path (str): Path to the image file for the chart.
            title (str): Title of the chart.
        """
        if title:
            self.section_title(title)
        try:
            # Open the image file
            with Image.open(chart_path) as img:
                # Save the image temporarily to disk
                with NamedTemporaryFile(delete=False, suffix=".png") as tmp_file:
                    temp_path = tmp_file.name
                    img.save(temp_path, format="PNG")
            
            # Embed the temporarily saved image into the PDF
            self.image(temp_path, x=10, w=190)
            self.ln(10)
            
            # Delete the temporary file after embedding
            os.remove(temp_path)
        except Exception as e:
            print(f"Error embedding chart from {chart_path}: {e}")


# Generate a preview
pdf = PDFReport()

# Add cover page
pdf.cover_page()

# Add a sample page for layout testing
pdf.add_page()
pdf.section_title("Sample Section Title")
pdf.section_body("This is a sample text to demonstrate the layout of a section body. "
                 "We are testing the alignment, font size, and spacing to ensure a professional appearance.")

# Save the preview
preview_pdf_path = "../outputs/report_preview.pdf"
pdf.output(preview_pdf_path)
print(f"Preview saved to {preview_pdf_path}")

# Step 4: Assemble the Report

In this step, we assemble the entire report by combining all sections into a single PDF:
1. Adding a professional cover page.
2. Summarizing insights by sentiment and visualizing sentiment distribution.
3. Providing the percentage breakdown of negative topics with an embedded chart.
4. Including problem descriptions for the top 3 worst topics.
5. Presenting consolidated findings and actionable recommendations.

The final report will be self-contained, with all charts embedded into the PDF.

In [ ]:
# Initialize the PDF report
pdf = PDFReport()

# Step 4.1: Add Cover Page
pdf.cover_page()

# Step 4.2: Add Summary of Sentiments
pdf.add_page()
pdf.section_title("Sentiment Analysis Summary")
pdf.section_body(f"""
    Number of Reviews by Sentiment:
    Positive: {report_data['review_counts']['positive']}
    Negative: {report_data['review_counts']['negative']}
    Neutral: {report_data['review_counts']['neutral']}
""")

# Step 4.3: Visualize and Add Sentiment Distribution
sentiment_chart_path = "../outputs/charts/sentiment_distribution.png"
pdf.embed_chart(sentiment_chart_path, title="Sentiment Distribution")

# Step 4.4: Visualize and Add Generalized Topic Percentages for Negative Sentiment
negative_topics_chart_path = "../outputs/charts/negative_topics_distribution.png"
pdf.embed_chart(negative_topics_chart_path, title="Percentage Distribution of Negative Topics")

# Step 4.5: Add Problem Descriptions for Top 3 Worst Topics
pdf.add_page()
pdf.section_title("Problem Analysis - Top 3 Worst Topics")

for problem in report_data["problems_summary"]:
    pdf.section_title(f"Topic: {problem['topic']}")
    pdf.section_body(problem["problem_description"])

# Step 4.6: Add Consolidated Findings and Recommendations
pdf.add_page()
pdf.section_title("Consolidated Findings and Recommendations")

# Add consolidated problem description
pdf.section_title("Consolidated Problem Description")
pdf.section_body(report_data["general_problem_description"])

# Add consolidated recommendations
pdf.section_title("Consolidated Recommendations")
for i, recommendation in enumerate(report_data["consolidated_recommendations"], 1):
    pdf.section_body(f"{i}. {recommendation}")

# Save the assembled report
final_pdf_path = "../outputs/final_report.pdf"
pdf.output(final_pdf_path)
print(f"Final report saved to {final_pdf_path}")